<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
dataa = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922")

Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(dataa.content, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
soup.title.text

'List of Falcon 9 and Falcon Heavy launches - Wikipedia'

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
#soup.find_all
#html_tables=soup.find_all("table", {"class": "wikitable"})
html_tables=soup.find_all("table")
print(f"Total tables found: {len(html_tables)}")

Total tables found: 25


In [16]:
html_tables[0]

<table class="col-begin" role="presentation">
<tbody><tr>
<td class="col-break">
<div class="mw-heading mw-heading3"><h3 id="Rocket_configurations">Rocket configurations</h3></div>
<div class="chart noresize" style="padding-top:10px;margin-top:1em;max-width:420px;">
<div style="position:relative;min-height:320px;min-width:420px;max-width:420px;">
<div style="float:right;position:relative;min-height:240px;min-width:320px;max-width:320px;border-left:1px black solid;border-bottom:1px black solid;">
<div style="position:absolute;left:3px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-print-color-adjust:exact;border:1px solid LightSteelBlue;border-bottom:none;overflow:hidden;" title="[[Falcon 9 v1.0]]: 2"></div>
<div style="position:absolute;left:55px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-print-color-adjust:exact;border:1px solid LightSteelBlue;border-bottom:none;overflow:hidden;" title="[[Fal

In [30]:
# Step 1: Find the "Past launches" <h2> heading
past_launches_h2 = None
for h2 in soup.find_all("h2"):
    if "Past launches" in h2.get_text(strip=True):
        past_launches_h2 = h2
        break

if not past_launches_h2:
    print("Main title 'Past launches' not found.")
else:
    print("Main Title:", past_launches_h2.get_text(strip=True))
    
    # Step 2: Start from the "Past launches" heading, find tables below it
    sibling = past_launches_h2.find_next_sibling()
    table_count = 0

    while sibling and sibling.name != "h2":
        if sibling.name == "table":
            table_count += 1
            # Try to find a caption or the nearest heading above it
            caption = sibling.find("caption")
            if caption:
                print(f"Table {table_count} Caption:", caption.get_text(strip=True))
            else:
                # Try previous siblings to see if any contain bolded text or year-range
                previous = sibling.find_previous_sibling()
                found_label = False
                while previous and previous.name not in ["h2", "table"]:
                    bold = previous.find("b")
                    if bold:
                        text = bold.get_text(strip=True)
                        if any(char.isdigit() for char in text):
                            print(f"Table {table_count} Subsection:", text)
                            found_label = True
                            break
                    previous = previous.find_previous_sibling()
                if not found_label:
                    print(f"Table {table_count} has no subsection title.")
        sibling = sibling.find_next_sibling()


Main Title: Past launches


Starting from the third table is our target table contains the actual launch records.


In [9]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [10]:
column_names = []
colmm = first_launch_table.find_all("th")
for colmm in colmm:
    namee = extract_column_from_header(colmm)
#    column_names = extract_column_from_header(colmm)
    if namee is not None and len(namee) > 0:
        column_names.append(namee)
    
print(column_names)

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [42]:
first_launch_table.find_all("th")

[<th scope="col">Flight No.
 </th>,
 <th scope="col">Date andtime ()
 </th>,
 <th scope="col"> 
 </th>,
 <th scope="col">Launch site
 </th>,
 <th scope="col">Payload
 </th>,
 <th scope="col">Payload mass
 </th>,
 <th scope="col">Orbit
 </th>,
 <th scope="col">Customer
 </th>,
 <th scope="col">Launchoutcome
 </th>,
 <th scope="col">
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">1
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">2
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">3
 </th>,
 <th rowspan="3" scope="row" style="text-align:center;">4
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">5
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">6
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">7
 </th>]

Check the extracted column names


In [11]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [15]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [24]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            launch_dict['Flight No.'] = flight_number
            datatimelist=date_time(row[0])
            
print(launch_dict)
            

{'Flight No.': '121', 'Launch site': [], 'Payload': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launch outcome': [], 'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': [], 'key<tr>\n<th rowspan="2" scope="row" style="text-align:center;">1\n</th>\n<td>4 June 2010,<br/>18:45\n</td>\n<td><a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">F9 v1.0</a><sup class="reference" id="cite_ref-MuskMay2012_13-0"><a href="#cite_note-MuskMay2012-13"><span class="cite-bracket">[</span>7<span class="cite-bracket">]</span></a></sup><br/>B0003.1<sup class="reference" id="cite_ref-block_numbers_14-0"><a href="#cite_note-block_numbers-14"><span class="cite-bracket">[</span>8<span class="cite-bracket">]</span></a></sup>\n</td>\n<td><a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">CCAFS</a>,<br/><a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">SLC-40</a>\n</td>\n<td><a href="/wiki/Dragon

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [27]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            launch_dict['Flight No.'] = flight_number
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'] = date
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'] = time
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
                launch_dict['Version Booster'] = bv
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch Site'] = launch_site
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'] = payload
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'] = payload_mass
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'] = orbit
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            #customer = row[6].a.string
            customer = row[6].a.string.strip() if row[6].a and row[6].a.string else row[6].text.strip()
            launch_dict['Customer'] = customer
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'] = launch_outcome
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster Landing'] = booster_landing
            #print(booster_landing)
            
print(launch_dict)
            

F9 v1.07B0003.18
F9 v1.07B0004.18
F9 v1.07B0005.18
F9 v1.07B0006.18
F9 v1.07B0007.18
F9 v1.17B10038
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 FT[
F9 v1.1[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT♺[
F9 FT[
F9 FT[
F9 FT[
F9 FTB1029.2195
F9 FT[
F9 FT[
F9 B4[
F9 FT[
F9 B4[
F9 B4[
F9 FTB1031.2220
F9 B4[
F9 FTB1035.2227
F9 FTB1036.2227
F9 B4[
F9 FTB1032.2245
F9 FTB1038.2268
F9 B4[
F9 B4B1041.2268
F9 B4B1039.2292
F9 B4[
F9 B5311B1046.1268
F9 B4B1043.2322
F9 B4B1040.2268
F9 B4B1045.2336
F9 B5
F9 B5349B1048[
F9 B5B1046.2354
F9 B5[
F9 B5B1048.2364
F9 B5B1047.2268
F9 B5B1046.3268
F9 B5[
F9 B5[
F9 B5B1049.2397
F9 B5B1048.3399
F9 B5[]413
F9 B5[
F9 B5B1049.3434
F9 B5B1051.2420
F9 B5B1056.2465
F9 B5B1047.3472
F9 B5
F9 B5[
F9 B5B1056.3482
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5[
F9 B5
F9 B5
F9 B5
F9 B5B1058.2544
F9 B5
F9 B5B1049.6544
F9 B5
F9 B5B1060.2563
F9 B5B1058.3565
F9 B5B1051.6568


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [34]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
df

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,...,"key<tr>\n<th rowspan=""2"" scope=""row"" style=""text-align:center;"">114\n</th>\n<td>23 April 2021<br/>9:49<sup class=""reference"" id=""cite_ref-658""><a href=""#cite_note-658""><span class=""cite-bracket"">[</span>646<span class=""cite-bracket"">]</span></a></sup>\n</td>\n<td><a href=""/wiki/Falcon_9_Block_5"" title=""Falcon 9 Block 5"">F9 B5</a> ♺<br/><a href=""/wiki/List_of_Falcon_9_first-stage_boosters#B1061"" title=""List of Falcon 9 first-stage boosters"">B1061.2</a><sup class=""reference"" id=""cite_ref-:6_659-0""><a href=""#cite_note-:6-659""><span class=""cite-bracket"">[</span>647<span class=""cite-bracket"">]</span></a></sup>\n</td>\n<td><a href=""/wiki/Kennedy_Space_Center"" title=""Kennedy Space Center"">KSC</a>,<br/><a href=""/wiki/Kennedy_Space_Center_Launch_Complex_39A"" title=""Kennedy Space Center Launch Complex 39A"">LC-39A</a>\n</td>\n<td><a href=""/wiki/SpaceX_Crew-2"" title=""SpaceX Crew-2"">Crew-2</a><br/>(<a class=""mw-redirect"" href=""/wiki/SpaceX_Crew_Dragon_Endeavour"" title=""SpaceX Crew Dragon Endeavour"">Crew Dragon C206.2 <i>Endeavour</i></a> ♺)\n</td>\n<td>~13,000 kg (29,000 lb)<sup class=""reference"" id=""cite_ref-660""><a href=""#cite_note-660""><span class=""cite-bracket"">[</span>648<span class=""cite-bracket"">]</span></a></sup>\n</td>\n<td><a class=""mw-redirect"" href=""/wiki/Low_Earth_Orbit"" title=""Low Earth Orbit"">LEO</a> (<a class=""mw-redirect"" href=""/wiki/ISS"" title=""ISS"">ISS</a>)\n</td>\n<td><a href=""/wiki/NASA"" title=""NASA"">NASA</a> (<a class=""mw-redirect"" href=""/wiki/ISS_Crew_Transportation_Services"" title=""ISS Crew Transportation Services"">CTS</a>)<sup class=""reference"" id=""cite_ref-CCD6_509-2""><a href=""#cite_note-CCD6-509""><span class=""cite-bracket"">[</span>497<span class=""cite-bracket"">]</span></a></sup>\n</td>\n<td class=""table-success"" style=""background: #9EFF9E; color:black; vertical-align: middle; text-align: center;"">Success\n</td>\n<td class=""table-success"" style=""background: #9EFF9E; color:black; vertical-align: middle; text-align: center;"">Success<br/><small><span class=""nowrap"">(drone ship)</span></small>\n</td></tr>","key<tr>\n<th rowspan=""2"" scope=""row"" style=""text-align:center;"">115\n</th>\n<td>29 April 2021<br/>03:44<sup class=""reference"" id=""cite_ref-663""><a href=""#cite_note-663""><span class=""cite-bracket"">[</span>651<span class=""cite-bracket"">]</span></a></sup>\n</td>\n<td><a href=""/wiki/Falcon_9_Block_5"" title=""Falcon 9 Block 5"">F9 B5</a> ♺<br/><a href=""/wiki/List_of_Falcon_9_first-stage_boosters#B1060"" title=""List of Falcon 9 first-stage boosters"">B1060.7</a><sup class=""reference"" id=""cite_ref-664""><a href=""#cite_note-664""><span class=""cite-bracket"">[</span>652<span class=""cite-bracket"">]</span></a></sup>\n</td>\n<td><a href=""/wiki/Cape_Canaveral_Space_Force_Station"" title=""Cape Canaveral Space Force Station"">CCSFS</a>,<br/><a href=""/wiki/Cape_Canaveral_Space_Launch_Complex_40"" title=""Cape Canaveral Space Launch Complex 40"">SLC-40</a>\n</td>\n<td><a href=""/wiki/Starlink"" title=""Starlink"">Starlink</a> 24 v1.0 (60 satellites)\n</td>\n<td>15,600 kg (34,400 lb)\n</td>\n<td><a href=""/wiki/Low_Earth_orbit"" title=""Low Earth orbit"">LEO</a>\n</td>\n<td><a href=""/wiki/SpaceX"" title=""SpaceX"">SpaceX</a>\n</td>\n<td class=""table-success"" style=""background: #9EFF9E; color:black; vertical-align: middle; text-align: center;"">Success\n</td>\n<td class=""table-success"" style=""background: #9EFF9E; color:black; vertical-align: middle; text-align: center;"">Success<br/><small><span class=""nowrap"">(drone ship)</span></small>\n</td></tr>","key<tr>\n<th rowspan=""2"" scope=""row"" style=""text-align:center;"">116\n</th>\n<td>4 May 2021<br/>19:01<sup class=""reference"" id=""cite_ref-666""><a href=""#cite_note-666""><span class=""cite-bracket"">[</span>654<span class="

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
